In [1]:
from typing import Type
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool
from langchain.agents import initialize_agent, AgentType
from langchain.utilities import DuckDuckGoSearchAPIWrapper
from pydantic import BaseModel, Field

llm = ChatOpenAI(temperature=0.1)


class StockMarketSymbolSearchToolArgsSchema(BaseModel):
    query: str = Field(description="The query you will search for")


class StockMarketSymbolSearchTool(BaseTool):
    name = "StockMarketSymbolSearchTool"
    description = """
    Use this tool to find the stock market symbol for a company.
    It takes a query as an argument.
    Example query: Stock Market Symbol for Apple Company
    """
    args_schema: Type[StockMarketSymbolSearchToolArgsSchema] = (
        StockMarketSymbolSearchToolArgsSchema
    )

    def _run(self, query):
        ddg = DuckDuckGoSearchAPIWrapper()
        return ddg.run(query)


agent = initialize_agent(
    llm=llm,
    verbose=True,
    agent=AgentType.OPENAI_FUNCTIONS,
    handle_parsing_errors=True,
    tools=[
        StockMarketSymbolSearchTool(),
    ],
)

prompt = "Give me information on Cloudflare's stock and help me analyze if it's a potential good investment. Also tell me what symbol does the stock have."

agent.invoke(prompt)



> Entering new AgentExecutor chain...

Invoking: `StockMarketSymbolSearchTool` with `{'query': 'Cloudflare stock'}`


Cloudflare Q2 Earnings Highlights: Revenue Beat, EPS Beat, Guidance Shines. Cloud connectivity company Cloudflare Inc NET reported second-quarter revenue of $401.0 million, up 30% year-over-year. The revenue beat a Street consensus estimate of $394.69 million, according to da... 5 weeks ago - Benzinga. Real-time Cloudflare stock price (NYSE: NET) plus other valuable data points like day range, year, stock analyst insights, related news and more. Cloudflare reported a strong Q2 with revenue and operating income beating expectations, and raised its full-year guidance. The company also announced a leadership change and highlighted its AI and developer growth. Cloudflare's stock has recently been incredibly volatile, ranging between approximately $30 and $80. The significant price movements have been in response to news and events affecting the broader tech industry. Clou

{'input': "Give me information on Cloudflare's stock and help me analyze if it's a potential good investment. Also tell me what symbol does the stock have.",
 'output': "Cloudflare Inc. (stock symbol: NET) reported strong Q2 earnings with revenue beating expectations and raised its full-year guidance. The stock has been volatile, ranging between approximately $30 and $80, with significant price movements in response to news and events affecting the broader tech industry.\n\nTo analyze if Cloudflare is a potential good investment, we can consider the following factors:\n1. **Financial Performance**: Look at the company's revenue growth, profitability, and cash flow.\n2. **Competitive Position**: Evaluate Cloudflare's position in the competitive cybersecurity industry.\n3. **Future Growth Potential**: Consider factors like market trends, product innovation, and expansion plans.\n4. **Risk Factors**: Assess risks such as competition, regulatory challenges, and market volatility.\n\nIt's i